In [ ]:
#RNN模型一般直接用来进行长期记忆计算
#LSTM模型代码  
#两层LSTM  一层全连接 ，输出为一个10000维的向量
#PTB数据集   完整代码如下
#参考https://blog.csdn.net/White_Idiot/article/details/78881261  
#http://dataunion.org/30472.html此网站极其详细介绍tf的RNN
import tensorflow as tf
from tensorflow.models.tutorials.rnn.ptb import reader
import numpy as np

#读文件
DATA_PATH = r'C:\Users\abc\Downloads\simple-examples\simple-examples\data'#存放的位置
train_data ,valid_data,test_data ,_= reader.ptb_raw_data(DATA_PATH)#每个单词转化成数字格式了
#print (len(train_data))#txt总共929589个单词  句子结束标识符为2

#参数设置
VOCAB_SIZE = 10000  #词典单词数

#训练超参数
TRAIN_BATCH_SIZE = 20 #batch大小   
LEARNING_RATE = 1.0   #学习率
TRAIN_NUM_STEP = 35  #截断长度     时间长度

#测试不需要截断
EVAL_BATCH_SIZE = 1 #batch大小
EVAL_NUM_STEP = 1  #截断长度

#神经网络参数
HIDDEN_SIZE = 200  #LSTM隐藏层节点
NUM_EPOCH = 20    #训练轮数
KEEP_PROB = 0.5   #节点不被droupout概率
MAX_GRAD_NORM = 5  #梯度膨胀参数
NUM_LAYERS = 2  #LSTM结构层数

# 通过PTBModel描述模型，方便维护循环神经网络中的状态
class PTBModel():
    def __init__(self, is_training, batch_size, num_steps):
        # 记录batch和截断长度
        self.batch_size = batch_size
        self.num_steps = num_steps

        # 定义输入层
        self.input_data = tf.placeholder(tf.int32, [batch_size, num_steps])

        # 定义预期输出
        self.targets = tf.placeholder(tf.int32, [batch_size, num_steps])

        # 定义LSTM为使用dropout的两层网络
        num_layer_size = [HIDDEN_SIZE, HIDDEN_SIZE]
        #lstm_cell = tf.contrib.rnn.BasicLSTMCell(HIDDEN_SIZE)
        lstm_cell = [tf.nn.rnn_cell.BasicLSTMCell(num_units=size) for size in num_layer_size]
        if is_training:
            for i in range(len(num_layer_size)):
                lstm_cell[i] = tf.contrib.rnn.DropoutWrapper(
                        lstm_cell[i], output_keep_prob=KEEP_PROB)
        cell = tf.nn.rnn_cell.MultiRNNCell(
                                       lstm_cell, state_is_tuple=True)
#注意使用多层cell时必须用下面的写法而不能直接简单地定义一个单层cell然后*num_layers，因为这样会导致各cell层共享权重
#tf.nn.rnn_cell.LSTMCell
        

        # 初始化state
        self.initial_state = cell.zero_state(batch_size, tf.float32)###？？？？？？？为什么是batch_size个

        # 将单词ID转为单词向量。每个单词都是HIDDEN_SIZE维
        embedding = tf.get_variable('embedding', [VOCAB_SIZE, HIDDEN_SIZE])

        # 将原本batch_size*num_steps的输入层转化为batch_size*num_steps*HIDDEN_SIZE的三维0/1矩阵
        inputs = tf.nn.embedding_lookup(embedding, self.input_data)#One-hot型的矩阵相乘，可以简化为查表操作

        # 只在训练时使用dropout
        if is_training:
            inputs = tf.nn.dropout(inputs, KEEP_PROB)

        # 定义输出列表
        outputs = []
        state = self.initial_state
        with tf.variable_scope('RNN'):
            for time_step in range(num_steps):
                if time_step > 0:
                    tf.get_variable_scope().reuse_variables()#管理传给get_variable()的变量名称的作用域  embedding使用
                cell_output, state = cell(inputs[:, time_step, :],
                                          state)  # 将当前时刻的数据和状态传入LSTM，需要state
                outputs.append(cell_output)  # 将当前输出加入输出列表

        # 将输出列表展开成[batch,hidden_size*num_steps]
        # 再reshape成[batch*num_steps,hidden_size]
        output = tf.reshape(tf.concat(outputs, 1), [-1, HIDDEN_SIZE])#concat指定维度重新拼接，本来是batch*hidden_size为元素的len为num_step的列表

        # 将输出传入全连接层，每个时刻的输出都是长度为VOCAB_SIZE的数组
        weight = tf.get_variable('weight', [HIDDEN_SIZE, VOCAB_SIZE])
        bias = tf.get_variable('bias', [VOCAB_SIZE])
        logits = tf.matmul(output, weight) + bias

        # 定义交叉熵损失函数，sequence_loss_by_example计算一个序列的交叉熵的和
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [logits],  # 预测结果
            [tf.reshape(self.targets, [-1])
             ],  # 预期结果。将[batch_size,num_steps]压缩成一维
            [tf.ones([batch_size * num_steps], dtype=tf.float32)
             ]  # 损失的权重。这里所有的权重都为1，表示不同batch和不同时刻的重要程度都一样
        )

        # 计算得到每个batch的平均损失
        self.cost = tf.reduce_sum(loss) / batch_size
        self.final_state = state

        # 只在训练时反向传播
        if not is_training:
            return
        trainable_variables = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(
            tf.gradients(self.cost, trainable_variables),#cost对所有训练变量求导
            MAX_GRAD_NORM)  # 控制梯度大小。避免梯度膨胀# 分界点为MAX_GRAD_NORM  小于则加，大于则减

        # 定义优化方法
        optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)

        # 定义训练步骤
        self.train_op = optimizer.apply_gradients(
            zip(grads, trainable_variables))


def run_epoch(session, model, data_queue, train_op, output_log, epoch_size):#model是一个类，方法和属性之间相互利用
    # 计算perplexity的辅助变量
    total_costs = 0.0
    iters = 0
    state = session.run(model.initial_state)#run lstm得到output_state
    #feed机制可以对已经赋值和未赋值（placeholder）都进行赋值
    # 使用当前数据训练或测试模型
    for step in range(epoch_size):
        # 生成输入和答案
        feed_dict = {}
        x, y = session.run(data_queue)
        feed_dict[model.input_data] = x
        feed_dict[model.targets] = y

        # 将状态转为字典
        for i, (c, h) in enumerate(model.initial_state):####len = batch  #每次迭代都把上过一次每层的statefeed过去，目的是复用LSTM模型那个类
            feed_dict[c] = state[i].c#输出
            feed_dict[h] = state[i].h#向下一个时间传递 直接feed给final_state，state可以得到output进而得到cost
        #print(len(feed_dict))输出是6  2个是input和target  剩下四个分别是每层的output和state
        #feed_dict 有数据队伍，目标队伍，LSTM模型两层的的输出和传递状态
        # 获取损失值和下一个状态
        cost, state, _ = session.run(
            [model.cost, model.final_state, train_op], feed_dict=feed_dict
        )  # 在当前batch上运行train_op并计算损失值。交叉熵损失函数计算的是下一个单词为给定单词的概率
        #feed output是给全连接神经网络
        total_costs += cost
        iters += model.num_steps

        # 训练时输出日志
        if output_log and step % 100 == 0:
            print('After %d steps,perplexity is %.3f' %
                  (step, np.exp(total_costs / iters)))

    return np.exp(total_costs / iters)

#一个epoch遍历所有训练数据，一个batch更新一次模型
def main(_):
    # 原始数据
    train_data, valid_data, test_data, _ = reader.ptb_raw_data(DATA_PATH)

    # 计算一个epoch需要训练的次数
    train_data_len = len(train_data)  # 数据集的大小
    train_batch_len = train_data_len // TRAIN_BATCH_SIZE  # batch的个数
    train_epoch_size = (train_batch_len - 1) // TRAIN_NUM_STEP  # 该epoch的训练次数

    valid_data_len = len(valid_data)
    valid_batch_len = valid_data_len // EVAL_BATCH_SIZE
    valid_epoch_size = (valid_batch_len - 1) // EVAL_NUM_STEP

    test_data_len = len(test_data)
    test_batch_len = test_data_len // EVAL_BATCH_SIZE
    test_epoch_size = (test_batch_len - 1) // EVAL_NUM_STEP
    
    # 定义初始化函数
    initializer = tf.random_uniform_initializer(-0.05, 0.05)

    # 定义训练用的模型
    with tf.variable_scope(
            'language_model', reuse=None, initializer=initializer):
        train_model = PTBModel(True, TRAIN_BATCH_SIZE, TRAIN_NUM_STEP)
     # 定义评估用的模型
    with tf.variable_scope(
            'language_model', reuse=True, initializer=initializer):
        eval_model = PTBModel(False, EVAL_BATCH_SIZE, EVAL_NUM_STEP)    
    
    # 生成数据队列，必须放在开启多线程之前
    train_queue = reader.ptb_producer(train_data, train_model.batch_size,
                                      train_model.num_steps)
    valid_queue = reader.ptb_producer(valid_data, eval_model.batch_size,
                                      eval_model.num_steps)
    test_queue = reader.ptb_producer(test_data, eval_model.batch_size,
                                     eval_model.num_steps)

    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        # 开启多线程从而支持ptb_producer()使用tf.train.range_input_producer()
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)#多线程只要指定sess和coord就行了

        # 使用训练数据训练模型
        for i in range(NUM_EPOCH):
            print('In iteration: %d' % (i + 1))
            run_epoch(sess, train_model, train_queue, train_model.train_op,
                      True, train_epoch_size)  # 训练模型
            valid_perplexity = run_epoch(sess, eval_model, valid_queue,
                                         tf.no_op(), False,
                                         valid_epoch_size)  # 使用验证数据评估模型
            print('Epoch: %d Validation Perplexity: %.3f' % (i + 1,
                                                             valid_perplexity))

        # 使用测试数据测试模型
        test_perplexity = run_epoch(sess, eval_model, test_queue,
                                    tf.no_op(), False, test_epoch_size)
        print('Test Perplexity: %.3f' % test_perplexity)

        # 停止所有线程
        coord.request_stop()
        coord.join(threads)


if __name__ == '__main__':
    tf.app.run()

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
In iteration: 1
After 0 steps,perplexity is 9970.101
After 100 steps,perplexity is 1404.397
After 200 steps,perplexity is 1065.800
Af

In [5]:
print(train_data[100])

431


In [11]:
#这里是测试新版本tensorflow函数
#按batch大小截断  函数为ptb_producer
#batch为4  截断长度为5
# result  = reader.ptb_producer(train_data,4,5)
# with tf.Session() as sess:
#     tf.global_variables_initializer().run()

#     # 开启多线程
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)

#     # 读取前两个batch，其中包括每个时刻的输入和对应的答案，ptb_producer()会自动迭代
#     for i in range(2):
#         x, y = sess.run(result)
#         print('x:', x)
#         print('y:', y)

#     # 关闭多线程
#     coord.request_stop()
#     coord.join(threads)



AttributeError: 'tuple' object has no attribute 'next'

In [3]:
#tf.nn.embedding_lookup测试
import tensorflow as tf
import numpy as np
input_ids = tf.placeholder(dtype=tf.int32, shape=[None,3])
embedding = a = np.asarray([[0.1, 0.2, 0.3], [1.1, 1.2, 1.3], [2.1, 2.2, 2.3], [3.1, 3.2, 3.3], [4.1, 4.2, 4.3]])
input_embedding = tf.nn.embedding_lookup(embedding, input_ids)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
# print(embedding.eval())
print(sess.run(input_embedding, feed_dict={input_ids: [[1, 2, 3],[ 3, 2, 1]]}))

[[[1.1 1.2 1.3]
  [2.1 2.2 2.3]
  [3.1 3.2 3.3]]

 [[3.1 3.2 3.3]
  [2.1 2.2 2.3]
  [1.1 1.2 1.3]]]


In [ ]:
#tf.clip_by_global_norm(t_list, clip_norm, use_norm=None, name=None) 
#通过权重梯度的总和的比率来截取多个张量的值。 
#t_list 是梯度张量， clip_norm 是截取的比率, 这个函数返回截取过的梯度张量和一个所有张量的全局范数。

#g = tf.gradients(y,x)  #交叉求导数

#RNNcell是可call的   例如我们有一个初始状态h0，还有输入x1，调用call(x1, h0)后就可以得到(output1, h1)
#再调用一次call(x2, h1)就可以得到(output2, h2)：
  
#ptb_producer()和slice切分数据集介绍的介绍，可以看出input应该是batch_size*epoch_size的  #https://blog.csdn.net/a343902152/article/details/54429096
#上面的网站还有个词汇表函数_build_vocab   用来构建词汇表
#tf.slice()到底怎么切  https://www.jianshu.com/p/71e6ef6c121b

In [7]:
train_data, valid_data, test_data, _ = reader.ptb_raw_data(DATA_PATH)

In [8]:
train_data[1]

9971